In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import optuna
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit
from xgboost import XGBRegressor
from scipy import stats
from utils import calculate_CI

In [ ]:
# Import hospitalization data
hosp = pd.read_csv("../data/COVID_hosp.csv")
hosp = hosp[hosp['geography'] == 'DE'][['date','total']]
hosp = hosp.rename(columns={'total': 'hosp'})
hosp['date'] = pd.to_datetime(hosp['date'])
hosp = hosp.set_index('date', drop=True)
# Calculate incidence per 100.000
hosp = hosp['hosp'] / (84357 / 100)

In [ ]:
# Import virus load data (interpolated)
virus = pd.read_excel("../data/amelag_aggregierte_kurve.xlsx")
virus = virus[["datum", "loess_vorhersage"]].dropna()
virus = virus.rename(columns={'datum': 'date', 'loess_vorhersage': 'virus'})
virus['date'] = pd.to_datetime(virus['date'])
virus = virus.set_index('date', drop=True)

In [ ]:
# Joining the two DataFrames on the 'date' column
hosp_virus = pd.merge(virus, hosp, on='date', how='inner')

In [ ]:
hosp_virus['hosp'] = np.log(hosp_virus['hosp'])

In [ ]:
hosp_virus['hosp_shifted'] = hosp_virus['hosp'].shift(-7)

In [ ]:
hosp_virus['hosp_diff'] = hosp_virus['hosp'].diff()
hosp_virus['hosp_diff_shifted'] = hosp_virus['hosp_diff'].shift(-7)

hosp_virus = hosp_virus.dropna()

In [ ]:
def split_dataframe(df, train_size, test_size, shift_size):
    """
    Split a given dataframe into smaller chunks with a constant train and test size.
    The chunk window is moved forward by a specified shift value.
    If the last test part of a chunk does not have the full test length, the chunk is discarded.

    Parameters:
    - df: pandas DataFrame, the dataframe to be split
    - train_size: int, the length of the train window
    - test_size: int, the length of the test window
    - shift_size: int, the value by which the chunk window moves forward

    Returns:
    - chunks: list of tuples, each tuple contains a train window and its corresponding test window
    """
    chunks = []

    # Determine the total length of the dataframe
    total_length = len(df)

    # Start index for the current chunk
    start_index = 0

    while start_index + train_size + test_size <= total_length:
        # Extract the train and test data for the current chunk
        train_data = df.iloc[start_index:start_index + train_size]
        test_data = df.iloc[start_index + train_size:start_index + train_size + test_size]

        # Add the train-test pair to the chunks list
        chunks.append((train_data, test_data))

        # Move the chunk window forward
        start_index += shift_size

    return chunks

In [ ]:
chunks = split_dataframe(hosp_virus, 70, 7, 7)

In [ ]:
def objective(trial, chunk_train_data, is_autoregressive: bool, model_type: str, cv_type: str) -> float:
    """
    Objective function for Optuna.
    Optimizes Hyperparameters for XBGoost or Random Forest model for a given Cross-Validation method.
    Data can be autoregressive or also include the waste water data.
    
    Args:
        trial (optuna.trial.Trial): Optuna's trial object used for optimization.
        chunk_train_data (pd.DataFrame): Training data chunk for the current iteration.
        is_autoregressive (bool): Flag indicating whether the data is autoregressive.
        model_type (str): Type of model to be optimized, either 'XGB' for XGBoost or 'RF' for Random Forest.
        cv_type (str): Type of cross-validation method to be used.

    Returns:
        float: Mean MAPE (Mean Absolute Percentage Error) across all cross-validation folds for the given hyperparameters.
    """
    # Define feature and target columns based on is_autoregressive flag
    TARGET_COLUMN = ['hosp_diff_shifted']
    if is_autoregressive:
        FEATURE_COLUMNS = ['hosp_diff']
    else:
        FEATURE_COLUMNS = ['virus', 'hosp_diff']


    # Define model parameters based on model_type
    if model_type == 'XGB':
        params = {
            'n_estimators': 200,
            'random_state': 42,
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5, log=True),
            'max_depth': trial.suggest_int('max_depth', 2, 12),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-4, 10, log=True),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-4, 10, log=True)
        }
        model = XGBRegressor(**params)
    elif model_type == 'RF':
        params = {
            'n_jobs': -1, # Utilize all available CPU cores
            'n_estimators': 200,
            'random_state': 42,
            'min_samples_split': trial.suggest_categorical('min_samples_split', [2, 4, 8, 16]),
            'min_samples_leaf': trial.suggest_categorical('min_samples_leaf', [1, 2, 4]),
        }
        model = RandomForestRegressor(**params)
        
    # Define CV-method to be used
    if cv_type == 'classical':
        splitter = TimeSeriesSplit(n_splits=5)
    if cv_type == 'blocked' or 'blocked_small':
        chunk_splits = split_dataframe(chunk_train_data, int(len(chunk_train_data)*0.2), int(len(chunk_train_data)*0.2), int(len(chunk_train_data)*0.1))
    if cv_type == 'weighted_linear' or cv_type == 'weighted_squared':
        splitter = TimeSeriesSplit(n_splits=5)
        weights = []
    if cv_type == 'k_fold':
        splitter = KFold(n_splits=5)
    if cv_type == 'no_cv':
        train_size = int(len(chunk_train_data) * 0.8)
        train_no_cv = chunk_train_data.iloc[:train_size]
        test_no_cv = chunk_train_data.iloc[train_size:]
        chunk_splits = [(train_no_cv, test_no_cv)]

    # Calculate average MAPE over all CV-folds (splits)
    split_mapes = []
    
    if cv_type in ['classical', 'weighted_linear', 'weighted_squared', 'k_fold']:
        for split_number, (train_idx, val_idx) in enumerate(splitter.split(chunk_train_data)):
            train_data = chunk_train_data.iloc[train_idx]
            val_data = chunk_train_data.iloc[val_idx]
    
            X_train, y_train = train_data[FEATURE_COLUMNS], train_data[TARGET_COLUMN]
            X_test, y_test = val_data[FEATURE_COLUMNS], np.exp(val_data['hosp_shifted'])
    
    
            y_train_df = y_train.copy()
            if model_type == 'RF':
                # Flatten the target variable if needed
                y_train = np.ravel(y_train)
    
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
    
            # Transform back
            initial_value = hosp_virus.loc[y_train_df.last_valid_index(), 'hosp_shifted']
            y_pred = initial_value + y_pred.cumsum()
            y_pred = np.exp(y_pred)
    
            mape = mean_absolute_error(y_test, y_pred) / abs(y_test.mean()) * 100
            split_mapes.append(mape)
    
            if cv_type == 'weighted_linear' or cv_type == 'weighted_squared':
                weights.append(len(train_idx))
    
    if cv_type in ['blocked', 'blocked_small', 'no_cv']:
        for split_number, (train_chunk, test_chunk) in enumerate(chunk_splits):
            X_train, y_train = train_chunk[FEATURE_COLUMNS], train_chunk[TARGET_COLUMN]
            X_test, y_test = test_chunk[FEATURE_COLUMNS], np.exp(test_chunk['hosp_shifted'])

            y_train_df = y_train.copy()
            if model_type == 'RF':
                # Flatten the target variable if needed
                y_train = np.ravel(y_train)

            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            # Transform back
            initial_value = hosp_virus.loc[y_train_df.last_valid_index(), 'hosp_shifted']
            y_pred = initial_value + y_pred.cumsum()
            y_pred = np.exp(y_pred)

            mape = mean_absolute_error(y_test, y_pred) / abs(y_test.mean()) * 100
            split_mapes.append(mape)


    # Return mean MAPE or weighted mean MAPE
    if cv_type == 'weighted_linear':
        linear_weights = [value / sum(weights) for value in weights]
        return np.average(split_mapes, weights=linear_weights)
    elif cv_type == 'weighted_squared':
        squared_weights = [value**2 / sum([value**2 for value in weights]) for value in weights]
        return np.average(split_mapes, weights=squared_weights)
    else:
        return np.mean(split_mapes)


In [ ]:
def optimize_chunks(is_autoregressive: bool, model_type: str, cv_type: str) -> None:
    """
    Optimizes model hyperparameters for each chunk of data.

    Args:
        is_autoregressive (bool): Flag indicating whether the data is autoregressive.
        model_type (str): Type of model to be optimized, either 'XGB' for XGBoost or 'RF' for Random Forest.
        cv_type (str): Type of cross-validation method to be used.
    """
    # Set the logging level of Optuna to ERROR
    optuna.logging.set_verbosity(optuna.logging.ERROR)
    
    # Reset the lists before starting a new optimization task
    global best_chunk_params, cv_mapes, test_mapes, test_maes  # Declare global variables
    best_chunk_params = []
    cv_mapes = []
    test_mapes = []
    test_maes = []

    for chunk_number, (chunk_train_data, chunk_test_data) in enumerate(chunks):
        print(f'CURRENT CHUNK: {chunk_number}/{len(chunks)}')
        
        if model_type == 'XGB':
            study = optuna.create_study(direction='minimize')
        if model_type == 'RF':
            # Define categorical parameters for Random Forest
            param_grid = {
                'min_samples_split': [2, 4, 8, 16],
                'min_samples_leaf': [1, 2, 4],
            }
            study = optuna.create_study(direction='minimize', sampler=optuna.samplers.GridSampler(param_grid))
            
        objective_func = lambda trial: objective(trial, chunk_train_data, is_autoregressive, model_type, cv_type)
        study.optimize(objective_func, n_trials=100) # Only used for TPE-Sampler 

        best_chunk_params.append(study.best_params)
        cv_mapes.append(study.best_value)

In [ ]:
def calculate_test_mapes(best_chunk_params, is_autoregressive: bool, model_type: str, cv_type: str, is_cross_modal: bool) -> None:
    """
    Calculate test MAPEs for each chunk using the best parameters found during optimization.

    Args:
        best_chunk_params (list): List of best parameters found for each chunk during optimization.
        is_autoregressive (bool): Flag indicating whether the data is autoregressive.
        model_type (str): Type of model to be used, either 'XGB' for XGBoost or 'RF' for Random Forest.
    """
    
    for chunk_number, (chunk_train_data, chunk_test_data) in enumerate(chunks):
        print(f'CURRENT TEST CHUNK: {chunk_number}')

        # Account for smaller train set in refit when cv method is blocked_small
        if cv_type == 'blocked_small':
            discard_size = int(len(chunk_train_data) * 0.8)
            chunk_train_data = chunk_train_data.iloc[discard_size:]

        best_params = best_chunk_params[chunk_number]  # Retrieve the best parameters for the current chunk

        # Define feature and target columns based on is_autoregressive flag
        TARGET_COLUMN = ['hosp_diff_shifted']
        if is_autoregressive:
            FEATURE_COLUMNS = ['hosp_diff']
        elif is_cross_modal and not is_autoregressive:
            print("cross_modal")
            FEATURE_COLUMNS = ['virus']
        else:
            FEATURE_COLUMNS = ['virus', 'hosp_diff']    

        # Define model based on model_type
        if model_type == 'XGB':
            model = XGBRegressor(**best_params)
        elif model_type == 'RF':
            model = RandomForestRegressor(**best_params)

        X_train = chunk_train_data[FEATURE_COLUMNS]
        y_train = chunk_train_data[TARGET_COLUMN]
        
        X_test = chunk_test_data[FEATURE_COLUMNS]
        y_test = np.exp(chunk_test_data['hosp_shifted'])  # Assuming the target column is 'hosp_shifted'
        
        if model_type == 'RF':
            # Flatten the target variable if needed
            y_train = np.ravel(y_train)

        # Fit the model with the best parameters
        model.fit(X_train, y_train)

        # Make predictions
        y_pred = model.predict(X_test)

        # Transform back
        initial_value = hosp_virus.loc[chunk_train_data.last_valid_index(), 'hosp_shifted']
        y_pred = initial_value + y_pred.cumsum()
        y_pred = np.exp(y_pred)
        # Calculate MAPE
        mape = mean_absolute_error(y_test, y_pred) / abs(y_test.mean()) * 100
        mae = mean_absolute_error(y_test, y_pred)

        # Append to test_mapes list
        test_mapes.append(mape)
        test_maes.append(mae)
    
    return(test_mapes, test_maes)
        

In [ ]:
# Choose between ['classical', 'weighted_linear', 'weighted_squared', 'blocked', 'blocked_small', 'k_fold', 'no_cv']
global CV_CHOICES
CV_CHOICES = ['classical', 'weighted_linear', 'weighted_squared', 'blocked', 'blocked_small', 'k_fold', 'no_cv']

In [ ]:
def optimize_and_evaluate_all():
    # Define all possible combinations of parameters
    is_autoregressive_options = [True, False]
    model_type_options = ['XGB','RF']
    cv_type_options = ['blocked_small']   # for paper only focus on classical ts cv
    is_cross_modal = [False]
    
    total_combinations = len(is_autoregressive_options) * len(model_type_options) * len(cv_type_options)
    current_combination = 0

    for is_autoregressive in is_autoregressive_options:
        for model_type in model_type_options:
            for cv_type in cv_type_options:
                current_combination += 1
                print(f"[{current_combination}/{total_combinations}] Optimizing and evaluating for: is_autoregressive={is_autoregressive}, model_type={model_type}, cv_type={cv_type}")
                
                # Optimize hyperparameters
                optimize_chunks(is_autoregressive=is_autoregressive, model_type=model_type, cv_type=cv_type)

                # Evaluate model
                test_mapes, test_maes = calculate_test_mapes(best_chunk_params, is_autoregressive=is_autoregressive, model_type=model_type, cv_type=cv_type, is_cross_modal=is_cross_modal)
                

                average_mape = np.round(np.mean(test_mapes),2)
                average_mae = np.round(np.mean(test_maes),2)
                median_mape = np.round(np.median(test_mapes),2)
                median_mae = np.round(np.median(test_maes),2)
                CI_mape = calculate_CI(test_mapes)
                CI_mae = calculate_CI(test_maes)
                
                output_name_metrics = "../output/"+model_type+"_"+str(is_autoregressive)+"_metrics.csv"
                output_name_summary = "../output/"+model_type+"_"+str(is_autoregressive)+"_summary.csv"
                
                metrics = pd.DataFrame({"MAPE":test_mapes, "MAE":test_maes})
                summary = pd.DataFrame({"Mean_MAPE":average_mape, "Median_MAPE":median_mape, "CI_low_MAPE":CI_mape[0],"CI_up_MAPE":CI_mape[1], 
                                        "Mean_MAE": average_mae, "Median_MAE":median_mae, "CI_low_MAE":CI_mae[0],"CI_up_MAE":CI_mae[1] }, index=[0])
                metrics.to_csv(output_name_metrics,index=False)
                summary.to_csv(output_name_summary,index=False)


In [ ]:
optimize_and_evaluate_all()

In [ ]:
# Evaluation for cross-modal only

def optimize_and_evaluate_cross_modal():
    # Define all possible combinations of parameters
    is_autoregressive_options = [False]
    model_type_options = ['XGB','RF']
    cv_type_options = ['blocked_small']   # for paper only focus on classical ts cv
    is_cross_modal = [True]
    
    total_combinations = len(is_autoregressive_options) * len(model_type_options) * len(cv_type_options)
    current_combination = 0

    for is_autoregressive in is_autoregressive_options:
        for model_type in model_type_options:
            for cv_type in cv_type_options:
                current_combination += 1
                print(f"[{current_combination}/{total_combinations}] Optimizing and evaluating for: is_autoregressive={is_autoregressive}, model_type={model_type}, cv_type={cv_type}")
                
                # Optimize hyperparameters
                optimize_chunks(is_autoregressive=is_autoregressive, model_type=model_type, cv_type=cv_type)

                # Evaluate model
                test_mapes, test_maes = calculate_test_mapes(best_chunk_params, is_autoregressive=is_autoregressive, model_type=model_type, cv_type=cv_type, is_cross_modal=is_cross_modal)
                

                average_mape = np.round(np.mean(test_mapes),2)
                average_mae = np.round(np.mean(test_maes),2)
                median_mape = np.round(np.median(test_mapes),2)
                median_mae = np.round(np.median(test_maes),2)
                CI_mape = calculate_CI(test_mapes)
                CI_mae = calculate_CI(test_maes)
                
                output_name_metrics = "../output/"+model_type+"_cross_modal_metrics.csv"
                output_name_summary = "../output/"+model_type+"_cross_modal_summary.csv"
                
                metrics = pd.DataFrame({"MAPE":test_mapes, "MAE":test_maes})
                summary = pd.DataFrame({"Mean_MAPE":average_mape, "Median_MAPE":median_mape, "CI_low_MAPE":CI_mape[0],"CI_up_MAPE":CI_mape[1], 
                                        "Mean_MAE": average_mae, "Median_MAE":median_mae, "CI_low_MAE":CI_mae[0],"CI_up_MAE":CI_mae[1] }, index=[0])
                metrics.to_csv(output_name_metrics,index=False)
                summary.to_csv(output_name_summary,index=False)


In [ ]:
optimize_and_evaluate_cross_modal()